# Frozen-in Magnetic Flux (Rm >> 1)

**Validation case**: Magnetic flux advection in the ideal MHD limit

## Learning Objectives

After completing this notebook, you will understand:

1. Alfvén's frozen-in flux theorem and its physical meaning
2. The magnetic Reynolds number and what Rm >> 1 means physically
3. How magnetic flux is conserved during plasma expansion
4. The analytic solution for flux conservation in radial expansion
5. How to validate numerical MHD solvers against flux conservation

---
## 1. Physics Background

### The Magnetic Induction Equation (Revisited)

The evolution of magnetic field in a conducting plasma is governed by:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B}) + \eta \nabla^2 \mathbf{B}$$

In the **ideal MHD limit** ($\eta \to 0$, or equivalently $R_m \gg 1$), this becomes:

$$\frac{\partial \mathbf{B}}{\partial t} = \nabla \times (\mathbf{v} \times \mathbf{B})$$

This seemingly simple equation has a profound consequence: **Alfvén's theorem**.

### Alfvén's Frozen-in Flux Theorem

**Theorem**: In ideal MHD, the magnetic flux through any surface moving with the plasma is conserved.

Mathematically, if $S(t)$ is a surface that moves with the fluid:

$$\frac{d}{dt} \int_{S(t)} \mathbf{B} \cdot d\mathbf{A} = 0$$

This means:
- Magnetic field lines are "frozen" into the plasma
- Field lines move with the fluid velocity
- Field lines cannot break or reconnect (in ideal MHD)

### Physical Picture

Imagine magnetic field lines as elastic threads sewn into a fluid:
- When the fluid stretches, the threads stretch with it
- When the fluid compresses, the threads bunch together
- The threads cannot slip through the fluid or break

### Our Test Case: Radial Expansion

We consider an annular plasma with:
- Uniform toroidal field $B_\phi = B_0$
- Uniform radial expansion velocity $v_r = v_0$

**Geometry**: Cylindrical coordinates $(r, \phi, z)$ with the field in the $\phi$ direction.

**Initial condition**: Uniform $B_\phi$ in the annular region $r_{\text{min}} < r < r_{\text{max}}$.

**Flux conservation**: The toroidal flux through a radial slice must be conserved:

$$\Phi = \int B_\phi \, dr = \text{constant}$$

### Analytic Solution

For a fluid element initially at radius $r_0$, its position at time $t$ is:

$$r(t) = r_0 + v_r t$$

Flux conservation for a thin annular shell gives:

$$B_\phi(r, t) \cdot r = B_0 \cdot r_0$$

Therefore:

$$B_\phi(t) = B_0 \frac{r_0}{r_0 + v_r t}$$

Key features:
- $B_\phi$ decreases as the plasma expands
- The decrease is inversely proportional to the radius
- Total flux remains constant
- Characteristic timescale: $\tau_{\text{adv}} = L / v_r$ where $L = r_{\text{max}} - r_{\text{min}}$

### Physical Significance

The frozen-in flux condition is fundamental to plasma confinement:

- **Magnetic confinement**: Plasma cannot easily cross field lines
- **Solar physics**: Field lines stretch with solar wind, creating the Parker spiral
- **FRC dynamics**: The closed field structure traps plasma inside
- **Flux conservation**: Sets limits on how fast configurations can change

When the frozen-in condition breaks down (magnetic reconnection), dramatic energy release can occur.

---
## 2. Configuration Setup

In [ ]:
# Standard imports
import jax
import jax.numpy as jnp
import jax.lax as lax
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

# jax-frc imports
from jax_frc.configurations import FrozenFluxConfiguration
from jax_frc.solvers.explicit import EulerSolver

# Notebook utilities
import sys
sys.path.insert(0, '.')
from _shared import plot_comparison, plot_error, compute_metrics, metrics_summary, animate_overlay

# Plotting style
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 11

In [ ]:
# === PHYSICS PARAMETERS ===

B_phi_0 = 1.0      # Initial toroidal field [T]
v_r = 0.1          # Radial expansion velocity [m/s]
eta = 1e-8         # Very small resistivity (Rm >> 1)

# === GRID PARAMETERS ===

nr = 64            # High radial resolution
nz = 8             # Minimal axial (uniform in z)
r_min = 0.2        # Inner radius [m]
r_max = 1.0        # Outer radius [m]

# === TIME PARAMETERS ===

# Advection timescale
L = r_max - r_min
tau_adv = L / v_r
print(f"Advection timescale: τ_adv = {tau_adv:.2f} s")

t_end = 0.5 * tau_adv    # Run for 0.5 advection times
dt = 1e-3 * tau_adv      # CFL-like timestep

print(f"Simulation time: t_end = {t_end:.2f} s")
print(f"Timestep: dt = {dt:.4f} s")
print(f"Expected steps: ~{int(t_end/dt)}")

In [ ]:
# Create configuration
config = FrozenFluxConfiguration(
    B_phi_0=B_phi_0,
    v_r=v_r,
    eta=eta,
    nr=nr,
    nz=nz,
    r_min=r_min,
    r_max=r_max,
)

# Build simulation components
geometry = config.build_geometry()
initial_state = config.build_initial_state(geometry)
model = config.build_model()
solver = EulerSolver()

print(f"Grid: {geometry.nr} × {geometry.nz} (r × z)")
print(f"Domain: r ∈ [{geometry.r_min:.2f}, {geometry.r_max:.2f}]")
print(f"Resolution: Δr = {geometry.dr:.4f}")
print(f"\nMagnetic Reynolds number: Rm = {config.magnetic_reynolds_number():.2e} (should be >> 1)")

### Visualize Initial Condition

In [ ]:
# Extract radial profile at mid-z
z_mid = geometry.nz // 2
r = np.array(geometry.r_grid[:, z_mid])

# Initial B_phi profile
Bphi_init = np.array(initial_state.B[:, z_mid, 1])  # B_phi is index 1 in cylindrical

# Initial v_r profile
vr_init = np.array(initial_state.v[:, z_mid, 0])  # v_r is index 0

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# B_phi
axes[0].plot(r, Bphi_init, 'b-', linewidth=2)
axes[0].axhline(B_phi_0, color='gray', linestyle='--', alpha=0.5, label=f'$B_0$ = {B_phi_0}')
axes[0].set_xlabel('r [m]', fontsize=12)
axes[0].set_ylabel('$B_\phi$ [T]', fontsize=12)
axes[0].set_title('Initial Toroidal Field', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim(r_min, r_max)

# v_r
axes[1].plot(r, vr_init, 'r-', linewidth=2)
axes[1].axhline(v_r, color='gray', linestyle='--', alpha=0.5, label=f'$v_r$ = {v_r}')
axes[1].set_xlabel('r [m]', fontsize=12)
axes[1].set_ylabel('$v_r$ [m/s]', fontsize=12)
axes[1].set_title('Radial Expansion Velocity', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(r_min, r_max)

plt.tight_layout()
plt.show()

# Compute initial flux
flux_init = np.trapezoid(Bphi_init, r)
print(f"Initial toroidal flux: Φ₀ = {flux_init:.4f} T·m")

---
## 3. Run Simulation

We evolve the plasma with uniform radial expansion. The toroidal field should decrease according to flux conservation.

In [ ]:
# Time stepping setup
n_steps = int(t_end / dt)
save_interval = max(1, n_steps // 50)  # ~50 snapshots

print(f"Running {n_steps} steps with dt = {dt:.4f}")
print(f"Saving every {save_interval} steps")

In [ ]:
# Run simulation using lax.scan for efficiency
@jax.jit
def scan_step(state, _):
    new_state = solver.step(state, dt, model, geometry)
    return new_state, new_state.B[:, z_mid, 1]

# Run simulation
print("Running simulation...")
final_state, Bphi_history = lax.scan(scan_step, initial_state, None, length=n_steps)

# Convert to numpy and subsample for plotting
Bphi_history = np.array(Bphi_history)[::save_interval]
times = np.arange(0, n_steps, save_interval) * dt

# Add final state if not included
Bphi_final = np.array(final_state.B[:, z_mid, 1])
if len(times) == 0 or times[-1] < t_end - dt:
    Bphi_history = np.vstack([Bphi_history, Bphi_final])
    times = np.append(times, final_state.time)

print(f"Simulation complete: t = {final_state.time:.4f} s")
print(f"Saved {len(times)} snapshots")

---
## 4. Compare with Analytic Solution

In [ ]:
# Compute analytic solution at final time
Bphi_analytic = np.array(config.analytic_solution(jnp.array(r), final_state.time))

# The simplified analytic solution gives uniform decrease
r_inner_final = r_min + v_r * final_state.time
print(f"Inner radius: r_min = {r_min:.2f} m → r(t) = {r_inner_final:.2f} m")
print(f"Expected B_phi ratio: {r_min / r_inner_final:.4f}")
print(f"Actual mean B_phi ratio: {np.mean(Bphi_final) / B_phi_0:.4f}")

In [ ]:
# Overlay comparison plot
fig, ax = plot_comparison(
    r, Bphi_final, np.full_like(r, Bphi_analytic),  # Analytic is uniform
    xlabel='r [m]', ylabel='$B_\phi$ [T]',
    title=f'Frozen Flux at t = {final_state.time:.4f} s',
    initial=Bphi_init
)
plt.show()

In [ ]:
# Flux conservation check
flux_final = np.trapezoid(Bphi_final, r)
flux_error = abs(flux_final - flux_init) / abs(flux_init)

print("Flux Conservation:")
print("=" * 40)
print(f"Initial flux: Φ₀ = {flux_init:.6f} T·m")
print(f"Final flux:   Φ  = {flux_final:.6f} T·m")
print(f"Relative error: {flux_error:.4%}")
print(f"Status: {'✓ PASS' if flux_error < 0.01 else '✗ FAIL'} (threshold: 1%)")

In [ ]:
# Compute validation metrics
metrics = compute_metrics(Bphi_final, np.full_like(Bphi_final, Bphi_analytic))
metrics['flux_conservation'] = flux_error

# Display with thresholds
thresholds = {
    'l2_error': 0.05,           # 5% relative L2 error
    'flux_conservation': 0.01,  # 1% flux conservation
}

print("\nValidation Metrics:")
print("=" * 40)
for name, value in metrics.items():
    threshold = thresholds.get(name, None)
    if threshold:
        status = '✓ PASS' if value <= threshold else '✗ FAIL'
        print(f"{name:20s}: {value:.4e}  (threshold: {threshold:.2e}) {status}")
    else:
        print(f"{name:20s}: {value:.4e}")

# Overall result
overall_pass = all(metrics[k] <= thresholds[k] for k in thresholds)
print("=" * 40)
print(f"Overall: {'✓ PASS' if overall_pass else '✗ FAIL'}")

---
## 5. Time Evolution Animation

In [ ]:
# Define analytic solution function for animation
def analytic_fn(r_arr, t):
    # Uniform value from simplified analytic solution
    val = config.analytic_solution(jnp.array(r_arr), t)
    return np.full_like(r_arr, float(val))

# Create animation
anim = animate_overlay(
    r, list(Bphi_history), analytic_fn, times,
    xlabel='r [m]', ylabel='$B_\phi$ [T]',
    title='Frozen Flux Advection',
    initial=Bphi_init,
    interval=100
)

HTML(anim.to_jshtml())

---
## 6. Interactive Exploration

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def explore_time(time_idx):
    """Show solution at a specific snapshot."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    t = times[time_idx]
    Bphi_num = Bphi_history[time_idx]
    Bphi_ana_val = float(config.analytic_solution(jnp.array(r), t))
    
    # Solution comparison
    axes[0].plot(r, Bphi_init, 'gray', linestyle=':', alpha=0.7, label='Initial')
    axes[0].plot(r, Bphi_num, 'b-', linewidth=2, label='Numerical')
    axes[0].axhline(Bphi_ana_val, color='orange', linestyle='--', linewidth=2, 
                    label=f'Analytic = {Bphi_ana_val:.3f}')
    axes[0].set_xlabel('r [m]', fontsize=12)
    axes[0].set_ylabel('$B_\phi$ [T]', fontsize=12)
    axes[0].set_title(f'Solution at t = {t:.4f} s', fontsize=12)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(r_min, r_max)
    axes[0].set_ylim(0, B_phi_0 * 1.1)
    
    # Flux over time
    flux_history = [np.trapezoid(Bphi_history[i], r) for i in range(len(times))]
    axes[1].plot(times, flux_history, 'g-', linewidth=2, label='Numerical')
    axes[1].axhline(flux_init, color='gray', linestyle='--', alpha=0.7, label='Initial')
    axes[1].axvline(t, color='red', linestyle=':', alpha=0.7, label=f't = {t:.3f}')
    axes[1].set_xlabel('Time [s]', fontsize=12)
    axes[1].set_ylabel('Flux Φ [T·m]', fontsize=12)
    axes[1].set_title('Flux Conservation', fontsize=12)
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print current flux
    current_flux = np.trapezoid(Bphi_num, r)
    flux_err = abs(current_flux - flux_init) / flux_init
    print(f"Current flux: {current_flux:.6f} T·m  |  Flux error: {flux_err:.4%}")

# Time slider
time_slider = widgets.IntSlider(
    value=0, min=0, max=len(times)-1, step=1,
    description='Snapshot:',
    continuous_update=False
)

ui = widgets.VBox([
    widgets.HTML('<h4>Time Evolution Explorer</h4>'),
    widgets.HTML('<p>Drag the slider to see the solution at different times. Watch B<sub>φ</sub> decrease while flux is conserved!</p>'),
    time_slider,
])

interactive_output = widgets.interactive_output(explore_time, {'time_idx': time_slider})

display(ui, interactive_output)

---
## Summary

This notebook demonstrated:

1. **Physics**: In the ideal MHD limit ($R_m \gg 1$), magnetic field is "frozen" into the plasma. Field lines move with the fluid and cannot slip through it.

2. **Alfvén's theorem**: Magnetic flux through any surface moving with the plasma is conserved:
   $$\frac{d\Phi}{dt} = 0$$

3. **Analytic solution**: For uniform radial expansion, $B_\phi$ decreases inversely with radius expansion:
   $$B_\phi(t) = B_0 \frac{r_0}{r_0 + v_r t}$$

4. **Validation**: The numerical solution conserves flux within the specified tolerance, confirming correct implementation of ideal MHD advection.

### Physical Significance

The frozen-in flux condition is fundamental to:
- **Magnetic confinement**: Plasma trapped inside closed field lines
- **FRC formation**: Flux trapping during field reversal
- **Solar wind**: Parker spiral from rotating sun + radial outflow
- **Magnetic reconnection**: When frozen-in breaks down, energy is released

### Try Next

- Increase `eta` to see flux leakage (transition toward $R_m \sim 1$)
- Increase `v_r` to see faster expansion
- Compare with the **magnetic diffusion** notebook to see the opposite limit ($R_m \ll 1$)